In [48]:
import pandas as pd

# Caminho do arquivo CSV original
input_file = 'corpus-musicas-brasileiras.csv'
output_file = 'forro_lyrics.csv'

# Ler o arquivo CSV e filtrar linhas com problemas
df = pd.read_csv(
        input_file, encoding_errors = 'ignore', skiprows = [ 10002, 10003,
                                                             11341, 11342,
                                                             19356, 19357,
                                                             19364, 19365,
                                                             140164, 140165,
                                                             140197, 140198,
                                                             182920, 182921,
                                                             182953, 182954 ] )

# Filtrar as linhas onde o Gênero é "forró"
filtered_df = df[ df[ 'Gênero Musical' ] == 'Forró' ]

# Obter a lista de todos os artistas únicos no DataFrame original (df)
lista_de_artistas: list[ str ] = filtered_df[ 'Artista' ].unique().tolist()

# Selecionar apenas as colunas "Artista" e "Letra da Música"
lyrics_df = filtered_df[ [ 'Artista', 'Nome da Música', 'Letra da Música' ] ]

# Dicionário para salvar todos os artistas e suas músicas
artist_dict: dict = { }

for artist in lista_de_artistas:
    # Nome do arquivo de saída
    output_file: str = f'artists/{artist}.csv'

    # Dicionário para salvar as músicas de um artista
    songs: dict[ str, list ] = { }

    for index, row in lyrics_df.iterrows():
        if row[ 'Artista' ] == artist:
            song_name: str = row[ 'Nome da Música' ]
            if song_name not in songs:
                letra: str = str( row[ 'Letra da Música' ] ).replace( '\n', ' ' ).replace( '  ', ' ' )
                songs[ song_name ] = [ artist, letra ]

    artist_dict[ artist ] = songs

    # Abrir o arquivo de saída no modo de escrita
    with open( output_file, 'w', encoding = 'utf-8' ) as outfile:
        # Iterar pelos itens do dicionário
        for song, items in songs.items():
            outfile.write( f"Artista: {items[ 0 ]}\n" )
            custom_name: str = str( song ).replace( ' - Ao Vivo', '' )
            outfile.write( f"Nome: {custom_name}\n" )
            outfile.write( f"Letra: {items[ 1 ]}\n" )
            outfile.write( f"\n" )

    print( f"Arquivo '{output_file}' criado com sucesso." )



Arquivo 'artists/João Gomes.csv' criado com sucesso.
Arquivo 'artists/Mari Fernandez.csv' criado com sucesso.
Arquivo 'artists/NATTAN.csv' criado com sucesso.
Arquivo 'artists/Kadu Martins.csv' criado com sucesso.
Arquivo 'artists/Rogerinho.csv' criado com sucesso.
Arquivo 'artists/Felipe Amorim.csv' criado com sucesso.
Arquivo 'artists/Luiz Gonzaga.csv' criado com sucesso.
Arquivo 'artists/Ary Lobo.csv' criado com sucesso.
Arquivo 'artists/Trio Nordestino.csv' criado com sucesso.
Arquivo 'artists/Gilberto Gil.csv' criado com sucesso.
Arquivo 'artists/Falamansa.csv' criado com sucesso.
Arquivo 'artists/Traia Véia.csv' criado com sucesso.
Arquivo 'artists/dj sv.csv' criado com sucesso.
Arquivo 'artists/Zé Vaqueiro.csv' criado com sucesso.
Arquivo 'artists/Manim Vaqueiro.csv' criado com sucesso.
Arquivo 'artists/Wesley Safadão.csv' criado com sucesso.
Arquivo 'artists/Os Barões Da Pisadinha.csv' criado com sucesso.
Arquivo 'artists/Henry Freitas.csv' criado com sucesso.
Arquivo 'artists/